In [65]:
import os
import requests
import pandas as pd
import json
from api_keys import census_api_key


In [72]:
census_api_key


'a59744beaa4abf3257d003cdee1ec1b2cae7a57b'

In [5]:
#define years
years = [2019, 2020, 2021, 2022, 2023]
variables1 = ["B19013_001E", #median household income
              "B25077_001E", #median home value
              "B25064_001E" #median gross rent
              ]
#create list to hold all the data returned
all_data = []

for year in years:
    #Set the base URL
    url = f"http://api.census.gov/data/{year}/acs/acs5" 
    #Set the Params
    params = {
    "get" : f"NAME,{','.join(variables1)}",   #requests the county name and median household income
    "for" : "county:005,051,067",   #specific counties Clackamas, Multnomah, Washington
    "in" : "state:41",         #Oregon
    "key" : census_api_key
}


# Make the API request
    response = requests.get(url, params=params)
    census_data = response.json()
    for row in census_data[1:]:
        row_with_year = row + [year]
        all_data.append(row_with_year)
    print(json.dumps(all_data, indent=4))


[
    [
        "Washington County, Oregon",
        "82215",
        "386600",
        "1359",
        "41",
        "067",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "80484",
        "395100",
        "1295",
        "41",
        "005",
        2019
    ],
    [
        "Multnomah County, Oregon",
        "69176",
        "386200",
        "1237",
        "41",
        "051",
        2019
    ]
]
[
    [
        "Washington County, Oregon",
        "82215",
        "386600",
        "1359",
        "41",
        "067",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "80484",
        "395100",
        "1295",
        "41",
        "005",
        2019
    ],
    [
        "Multnomah County, Oregon",
        "69176",
        "386200",
        "1237",
        "41",
        "051",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "82911",
        "421100",
        "1356",
        "41",
        "005",
        2020
    ],

In [6]:
column_header = census_data[0] + ["Year"]
column_header

['NAME',
 'B19013_001E',
 'B25077_001E',
 'B25064_001E',
 'state',
 'county',
 'Year']

In [7]:
rows = all_data[1]
rows

['Clackamas County, Oregon', '80484', '395100', '1295', '41', '005', 2019]

In [32]:
census_data_df = pd.DataFrame(all_data, columns=column_header)
census_data_df.head()


,NAME,B19013_001E,B25077_001E,B25064_001E,state,county,Year
0,"Washington County, Oregon",82215,386600,1359,41,067,2019
1,"Clackamas County, Oregon",80484,395100,1295,41,005,2019
2,"Multnomah County, Oregon",69176,386200,1237,41,051,2019
3,"Clackamas County, Oregon",82911,421100,1356,41,005,2020
4,"Multnomah County, Oregon",71425,410800,1309,41,051,2020


In [114]:
census_data_df = census_data_df.rename(columns={'NAME': 'County', 'B19013_001E': 'Median Household Income',  'B25077_001E': 'Median Home Value',
              'B25064_001E': 'Median Gross Rent', 'state': 'State Code', 'county': 'County Code'})
cleanded_median_income_df = census_data_df.drop(columns=["State Code", "County Code"])
cleanded_median_income_df = cleanded_median_income_df.set_index(["County", "Year"])
cleanded_median_income_df.head()


,,Median Household Income,Median Home Value,Median Gross Rent
County,Year,,,
"Washington County, Oregon",2019,82215,386600,1359
"Clackamas County, Oregon",2019,80484,395100,1295
"Multnomah County, Oregon",2019,69176,386200,1237
"Clackamas County, Oregon",2020,82911,421100,1356
"Multnomah County, Oregon",2020,71425,410800,1309


In [115]:
#Export to CSV file
cleanded_median_income_df.to_csv("output_file/income_vs_home_rent.csv", index=False)


**Demographic Profile of Population in Clackamas, Multnomah, and Washington County**


In [126]:
# Define the variables
variables = [
    "B01003_001E",  # Total population
    "B01001_002E",  #Total male population
    "B01001_026E", #Total Female population
    "B02001_002E", #White alone
    "B02001_003E", #African American Alone
    "B02001_005E", #Asian alone
    "B02001_008E", #2 or more races
    "B03002_002E", #Hispanic
]

years = [2019, 2020, 2021, 2022, 2023]
sex_race_list= []

#Loop through the years
for year in years:
    #Set the base URL
    url = f"http://api.census.gov/data/{year}/acs/acs5" 
    #Set the Params
    params = {
    "get" : f"NAME,{','.join(variables)}",  #requests a string a variables 
    "for" : "county:005,051,067",   #specific counties Clackamas, Multnomah, Washington
    "in" : "state:41",         #Oregon
    "key" : census_api_key
}

# Make the API request
    response = requests.get(url, params=params)
    sex_race_data = response.json()
    for row in sex_race_data[1:]:
        row_with_year = row + [year]
        sex_race_list.append(row_with_year)
    print(json.dumps(sex_race_list, indent=4))

[
    [
        "Washington County, Oregon",
        "589481",
        "291701",
        "297780",
        "447201",
        "11897",
        "62545",
        "31984",
        "491280",
        "41",
        "067",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "410463",
        "202194",
        "208269",
        "359191",
        "3865",
        "17543",
        "16885",
        "374797",
        "41",
        "005",
        2019
    ],
    [
        "Multnomah County, Oregon",
        "804606",
        "398513",
        "406093",
        "627124",
        "43988",
        "59715",
        "43346",
        "711330",
        "41",
        "051",
        2019
    ]
]
[
    [
        "Washington County, Oregon",
        "589481",
        "291701",
        "297780",
        "447201",
        "11897",
        "62545",
        "31984",
        "491280",
        "41",
        "067",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "410463",
      

In [127]:
#create a Dataframe and add a year column
columns = sex_race_data[0] + ["Year"]
rows = sex_race_list[1]

sex_race_df = pd.DataFrame(sex_race_list, columns=columns)
sex_race_df.head()


,NAME,B01003_001E,B01001_002E,B01001_026E,B02001_002E,B02001_003E,B02001_005E,B02001_008E,B03002_002E,state,county,Year
0,"Washington County, Oregon",589481,291701,297780,447201,11897,62545,31984,491280,41,067,2019
1,"Clackamas County, Oregon",410463,202194,208269,359191,3865,17543,16885,374797,41,005,2019
2,"Multnomah County, Oregon",804606,398513,406093,627124,43988,59715,43346,711330,41,051,2019
3,"Clackamas County, Oregon",415084,204887,210197,355937,3648,18497,22928,378244,41,005,2020
4,"Multnomah County, Oregon",809869,401083,408786,615909,44160,62827,53217,714437,41,051,2020


In [130]:
#Rename Columns 
renamed_sex_race_df = sex_race_df.rename(columns={
    "NAME": "County",
    "B01003_001E": "Total Population",
    "B01001_002E": "Total Male Population",
    "B01001_026E": "Total Female Population",
    "B02001_002E": "White Alone",
    "B02001_003E": "African American Alone",
    "B02001_005E": "Asian alone",
    "B02001_008E": "2 or more races",
    "B03002_002E": "Hispanic"})
# renamed_sex_race_df
drop_sex_race = renamed_sex_race_df.drop(columns=["state", "county"])
cleanded_sex_race_df = drop_sex_race.set_index(["County", "Year"])
cleanded_sex_race_df.head()

,,Total Population,Total Male Population,Total Female Population,White Alone,African American Alone,Asian alone,2 or more races,Hispanic
County,Year,,,,,,,,
"Washington County, Oregon",2019,589481,291701,297780,447201,11897,62545,31984,491280
"Clackamas County, Oregon",2019,410463,202194,208269,359191,3865,17543,16885,374797
"Multnomah County, Oregon",2019,804606,398513,406093,627124,43988,59715,43346,711330
"Clackamas County, Oregon",2020,415084,204887,210197,355937,3648,18497,22928,378244
"Multnomah County, Oregon",2020,809869,401083,408786,615909,44160,62827,53217,714437


In [131]:
#Export to CSV file
cleanded_sex_race_df.to_csv("output_file/demographics.csv", index=True)

**Population Counts per Age Group, Home Ownership per age group, and Renters per age group Data**

In [15]:
#AGE Groups 
# Define the Age variables
age_variables = [
    "B01001_008E", #Males 20-24
    "B01001_009E", #Males 25-29
    "B01001_012E", #Males 30-34
    "B01001_013E", #Males 35-39
    "B01001_014E", #Male 40–44
    "B01001_015E", #Male 45–49
    "B01001_016E", #Male 50–54
    "B01001_032E", #Females 20-24
    "B01001_033E", #Females 25-29
    "B01001_036E", #Females 30-34
    "B01001_037E",  #Females 35-39
    "B01001_038E", #Female 40–44
    "B01001_039E", #Female 45–49
    "B01001_040E", #Female aged 50–54
]


years = [2019, 2020, 2021, 2022, 2023]
age_list= []

#Loop through the years
for year in years:
    #Set the base URL
    url = f"http://api.census.gov/data/{year}/acs/acs5" 
    #Set the Params
    params = {
    "get" : f"NAME,{','.join(age_variables)}",  #requests a string a variables 
    "for" : "county:005,051,067",   #specific counties Clackamas, Multnomah, Washington
    "in" : "state:41",         #Oregon
    "key" : census_api_key
}
    
# Make the API request
    response = requests.get(url, params=params)
    age_data = response.json()
    for row in age_data[1:]:
        row_with_year = row + [year]
        age_list.append(row_with_year)
    print(json.dumps(age_list, indent=4)) 

[
    [
        "Washington County, Oregon",
        "3305",
        "3384",
        "24233",
        "22930",
        "21787",
        "20696",
        "18561",
        "2700",
        "3579",
        "23179",
        "22629",
        "21325",
        "20522",
        "18710",
        "41",
        "067",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "2698",
        "2292",
        "12608",
        "13250",
        "13336",
        "13708",
        "13864",
        "2360",
        "1708",
        "12989",
        "14067",
        "13043",
        "14071",
        "14066",
        "41",
        "005",
        2019
    ],
    [
        "Multnomah County, Oregon",
        "4127",
        "4343",
        "38327",
        "36108",
        "30939",
        "28881",
        "25023",
        "4902",
        "4395",
        "38368",
        "36833",
        "27928",
        "27542",
        "24090",
        "41",
        "051",
        2019
    ]
]
[
    [
        "Wash

## Ages, Homeownership and Renters per age range

In [36]:
#Convert Age Data to Dataframe
columns = age_data[0] + ["Year"]
rows = age_data[1]

age_df = pd.DataFrame(age_list, columns=columns)
age_df

,NAME,B01001_008E,B01001_009E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_032E,B01001_033E,B01001_036E,B01001_037E,B01001_038E,B01001_039E,B01001_040E,state,county,Year
0,"Washington County, Oregon",3305,3384,24233,22930,21787,20696,18561,2700,3579,23179,22629,21325,20522,18710,41,067,2019
1,"Clackamas County, Oregon",2698,2292,12608,13250,13336,13708,13864,2360,1708,12989,14067,13043,14071,14066,41,005,2019
2,"Multnomah County, Oregon",4127,4343,38327,36108,30939,28881,25023,4902,4395,38368,36833,27928,27542,24090,41,051,2019
3,"Clackamas County, Oregon",2469,2224,12918,12958,14416,13727,13734,2442,1995,12886,14067,13775,14042,13749,41,005,2020
4,"Multnomah County, Oregon",3925,4752,38935,35750,32076,29278,25266,4851,4388,38842,36094,29799,27828,24257,41,051,2020
5,"Washington County, Oregon",3211,3501,24593,23964,21479,20822,18604,2618,3527,23334,22676,21874,20668,18678,41,067,2020
6,"Clackamas County, Oregon",2240,2299,13062,13922,14550,13977,13950,2272,1744,13004,14432,14123,13881,13748,41,005,2021
7,"Multnomah County, Oregon",3949,5134,38961,36664,32853,29723,26203,4696,4548,38893,35990,30925,27795,24562,41,051,2021
8,"Washington County, Oregon",3309,3106,24548,24412,22300,21064,19034,3043,3201,23306,22351,22540,20553,18690,41,067,2021
9,"Clackamas County, Oregon",2136,2341,13232,13886,15140,13927,14005,2429,1847,13009,14178,14609,13719,13552,41,005,2022


In [37]:
age_df.dtypes


NAME           object
B01001_008E    object
B01001_009E    object
B01001_012E    object
B01001_013E    object
B01001_014E    object
B01001_015E    object
B01001_016E    object
B01001_032E    object
B01001_033E    object
B01001_036E    object
B01001_037E    object
B01001_038E    object
B01001_039E    object
B01001_040E    object
state          object
county         object
Year            int64
dtype: object

In [38]:
age_df["Ages 20-34"] = age_df[["B01001_008E", "B01001_009E", "B01001_012E",  "B01001_032E", "B01001_033E", "B01001_036E",]].astype(int).sum(axis=1)
age_df["Ages 35-44"] = age_df[["B01001_013E", "B01001_014E","B01001_037E", "B01001_038E"]].astype(int).sum(axis=1)
age_df["Ages 45-54"] = age_df[[ "B01001_015E", "B01001_016E","B01001_039E", "B01001_040E"]].astype(int).sum(axis=1)
age_df = age_df[["NAME", "Year", "Ages 20-34", "Ages 35-44", "Ages 45-54"]]
age_df.head()


,NAME,Year,Ages 20-34,Ages 35-44,Ages 45-54
0,"Washington County, Oregon",2019,60380,88671,78489
1,"Clackamas County, Oregon",2019,34655,53696,55709
2,"Multnomah County, Oregon",2019,94462,131808,105536
3,"Clackamas County, Oregon",2020,34934,55216,55252
4,"Multnomah County, Oregon",2020,95693,133719,106629


In [39]:
#Set Housing Variables
housing_variables = [
    "B25007_003E",  # Owner-occupied, aged 25–34
    "B25007_004E",  # Owner-occupied, aged 35–44
    "B25007_005E",  # Owner-occupied, aged 45–54
    "B25007_009E",  # Renter-occupied, aged 25–34
    "B25007_010E",  # Renter-occupied, aged 35–44
    "B25007_011E",  # Renter-occupied, aged 45–54
]

years = [2019, 2020, 2021, 2022, 2023]
housing_list= []

#Loop through the years
for year in years:
    #Set the base URL
    url = f"http://api.census.gov/data/{year}/acs/acs5" 
    #Set the Params
    params = {
    "get" : f"NAME,{','.join(housing_variables)}",  #requests a string a variables 
    "for" : "county:005,051,067",   #specific counties Clackamas, Multnomah, Washington
    "in" : "state:41",         #Oregon
    "key" : census_api_key
    }
    # Make the API request
    response = requests.get(url, params=params)
    housing_data = response.json()
    for row in housing_data[1:]:
        row_with_year = row + [year]
        housing_list.append(row_with_year)
    print(json.dumps(housing_list, indent=4)) 


[
    [
        "Washington County, Oregon",
        "568",
        "14042",
        "26672",
        "22539",
        "9157",
        "2971",
        "41",
        "067",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "392",
        "9226",
        "18341",
        "22288",
        "9267",
        "2728",
        "41",
        "005",
        2019
    ],
    [
        "Multnomah County, Oregon",
        "617",
        "17602",
        "38329",
        "29506",
        "12122",
        "4292",
        "41",
        "051",
        2019
    ]
]
[
    [
        "Washington County, Oregon",
        "568",
        "14042",
        "26672",
        "22539",
        "9157",
        "2971",
        "41",
        "067",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "392",
        "9226",
        "18341",
        "22288",
        "9267",
        "2728",
        "41",
        "005",
        2019
    ],
    [
        "Multnomah County, Oregon",
       

In [40]:
#Convert Data to Dataframe
columns = housing_data[0] + ["Year"]
rows = housing_data[1]

housing_df = pd.DataFrame(housing_list, columns=columns)
housing_df

,NAME,B25007_003E,B25007_004E,B25007_005E,B25007_009E,B25007_010E,B25007_011E,state,county,Year
0,"Washington County, Oregon",568,14042,26672,22539,9157,2971,41,067,2019
1,"Clackamas County, Oregon",392,9226,18341,22288,9267,2728,41,005,2019
2,"Multnomah County, Oregon",617,17602,38329,29506,12122,4292,41,051,2019
3,"Clackamas County, Oregon",494,9116,19354,22904,9457,3094,41,005,2020
4,"Multnomah County, Oregon",597,17728,38400,31387,12380,4804,41,051,2020
5,"Washington County, Oregon",547,13815,26270,23554,9762,3198,41,067,2020
6,"Clackamas County, Oregon",475,9077,19987,23679,9698,2990,41,005,2021
7,"Multnomah County, Oregon",525,18170,39030,32856,12494,4481,41,051,2021
8,"Washington County, Oregon",595,13980,26739,23762,10232,2953,41,067,2021
9,"Clackamas County, Oregon",511,9222,20190,23810,10531,3165,41,005,2022


In [41]:
housing_df = housing_df.rename(columns={
    "B25007_003E": "Owner-occupied, ages 25-34",
    "B25007_004E": "Owner-occupied, aged 35-44",
    "B25007_005E": "Owner-occupied, aged 45-54",
    "B25007_009E": "Renter-occupied, aged 25-34",
    "B25007_010E": "Renter-occupied, aged 35-44",
    "B25007_011E": "Renter-occupied, aged 45-54"})
housing_df

,NAME,"Owner-occupied, ages 25-34","Owner-occupied, aged 35-44","Owner-occupied, aged 45-54","Renter-occupied, aged 25-34","Renter-occupied, aged 35-44","Renter-occupied, aged 45-54",state,county,Year
0,"Washington County, Oregon",568,14042,26672,22539,9157,2971,41,067,2019
1,"Clackamas County, Oregon",392,9226,18341,22288,9267,2728,41,005,2019
2,"Multnomah County, Oregon",617,17602,38329,29506,12122,4292,41,051,2019
3,"Clackamas County, Oregon",494,9116,19354,22904,9457,3094,41,005,2020
4,"Multnomah County, Oregon",597,17728,38400,31387,12380,4804,41,051,2020
5,"Washington County, Oregon",547,13815,26270,23554,9762,3198,41,067,2020
6,"Clackamas County, Oregon",475,9077,19987,23679,9698,2990,41,005,2021
7,"Multnomah County, Oregon",525,18170,39030,32856,12494,4481,41,051,2021
8,"Washington County, Oregon",595,13980,26739,23762,10232,2953,41,067,2021
9,"Clackamas County, Oregon",511,9222,20190,23810,10531,3165,41,005,2022


In [42]:
merged_df = pd.merge(age_df,housing_df, on=("NAME", "Year"))
merged_df

,NAME,Year,Ages 20-34,Ages 35-44,Ages 45-54,"Owner-occupied, ages 25-34","Owner-occupied, aged 35-44","Owner-occupied, aged 45-54","Renter-occupied, aged 25-34","Renter-occupied, aged 35-44","Renter-occupied, aged 45-54",state,county
0,"Washington County, Oregon",2019,60380,88671,78489,568,14042,26672,22539,9157,2971,41,067
1,"Clackamas County, Oregon",2019,34655,53696,55709,392,9226,18341,22288,9267,2728,41,005
2,"Multnomah County, Oregon",2019,94462,131808,105536,617,17602,38329,29506,12122,4292,41,051
3,"Clackamas County, Oregon",2020,34934,55216,55252,494,9116,19354,22904,9457,3094,41,005
4,"Multnomah County, Oregon",2020,95693,133719,106629,597,17728,38400,31387,12380,4804,41,051
5,"Washington County, Oregon",2020,60784,89993,78772,547,13815,26270,23554,9762,3198,41,067
6,"Clackamas County, Oregon",2021,34621,57027,55556,475,9077,19987,23679,9698,2990,41,005
7,"Multnomah County, Oregon",2021,96181,136432,108283,525,18170,39030,32856,12494,4481,41,051
8,"Washington County, Oregon",2021,60513,91603,79341,595,13980,26739,23762,10232,2953,41,067
9,"Clackamas County, Oregon",2022,34994,57813,55203,511,9222,20190,23810,10531,3165,41,005


In [43]:
cleaned_house_age_df = merged_df.drop(columns=['state', 'county'])
cleaned_house_age_df = cleaned_house_age_df.rename(columns={"NAME": "County"})
cleaned_house_age_df.head()

,County,Year,Ages 20-34,Ages 35-44,Ages 45-54,"Owner-occupied, ages 25-34","Owner-occupied, aged 35-44","Owner-occupied, aged 45-54","Renter-occupied, aged 25-34","Renter-occupied, aged 35-44","Renter-occupied, aged 45-54"
0,"Washington County, Oregon",2019,60380,88671,78489,568,14042,26672,22539,9157,2971
1,"Clackamas County, Oregon",2019,34655,53696,55709,392,9226,18341,22288,9267,2728
2,"Multnomah County, Oregon",2019,94462,131808,105536,617,17602,38329,29506,12122,4292
3,"Clackamas County, Oregon",2020,34934,55216,55252,494,9116,19354,22904,9457,3094
4,"Multnomah County, Oregon",2020,95693,133719,106629,597,17728,38400,31387,12380,4804


In [44]:
cleaned_house_age_df.set_index(["County", "Year"])


,,Ages 20-34,Ages 35-44,Ages 45-54,"Owner-occupied, ages 25-34","Owner-occupied, aged 35-44","Owner-occupied, aged 45-54","Renter-occupied, aged 25-34","Renter-occupied, aged 35-44","Renter-occupied, aged 45-54"
County,Year,,,,,,,,,
"Washington County, Oregon",2019,60380,88671,78489,568,14042,26672,22539,9157,2971
"Clackamas County, Oregon",2019,34655,53696,55709,392,9226,18341,22288,9267,2728
"Multnomah County, Oregon",2019,94462,131808,105536,617,17602,38329,29506,12122,4292
"Clackamas County, Oregon",2020,34934,55216,55252,494,9116,19354,22904,9457,3094
"Multnomah County, Oregon",2020,95693,133719,106629,597,17728,38400,31387,12380,4804
"Washington County, Oregon",2020,60784,89993,78772,547,13815,26270,23554,9762,3198
"Clackamas County, Oregon",2021,34621,57027,55556,475,9077,19987,23679,9698,2990
"Multnomah County, Oregon",2021,96181,136432,108283,525,18170,39030,32856,12494,4481
"Washington County, Oregon",2021,60513,91603,79341,595,13980,26739,23762,10232,2953


In [111]:
#Export CSV file
cleaned_house_age_df.to_csv("output_file/housing_and_age.csv", index=True)

# Income Based on Age and Education Level

In [77]:
#Set Housing Variables
income_variables = [
    "B19013_001E", #Median income for all households
    "B19049_002E", #Median income for aged 25 and under
    "B19049_003E", #Median income ages 25 t0 44
    "B19049_004E", #Median inome ages 45-64
    "B19049_005E", #Median income ages 65 +
    "B20004_001E", #Median earning for all educaiton levels
    "B20004_002E", #Median earning for less than high school grad
    "B20004_003E", #Median earning for high school grad
    "B20004_004E", #Median earning for some college or associates degree
    "B20004_005E", #Median earning for bachelor's degree
    "B20004_006E", #Median earning for graduate or professional degree
    "B19001_007E", #income range $30,000 to $34,999
    "B19001_008E", #income range $35,000 to $39,999
    "B19001_009E", #income range$40,000 to $44,999
    "B19001_010E", #income $45,000 to $49,999
    "B19001_011E", #income $50,000 to $59,999
    "B19001_012E", #income $60,000 to $74,999
    "B19001_013E", #income $75,000 to $99,999
    "B19001_014E", #income $100,000 to $124,999
    "B19001_015E",	#income $125,000 to $149,999
    "B19001_016E", #income	$150,000 to $199,999
    "B19001_017E" #income $200,000 or more
    ]

years = [2019, 2020, 2021, 2022, 2023]
income_list= []

#Loop through the years
for year in years:
    #Set the base URL
    url = f"http://api.census.gov/data/{year}/acs/acs5" 
    #Set the Params
    params = {
    "get" : f"NAME,{','.join(income_variables)}",  #requests a string a variables 
    "for" : "county:005,051,067",   #specific counties Clackamas, Multnomah, Washington
    "in" : "state:41",         #Oregon
    "key" : census_api_key
}
# Make the API request
    response = requests.get(url, params=params)
    income_data = response.json()
    for row in income_data[1:]:
        row_with_year = row + [year]
        income_list.append(row_with_year)
    print(json.dumps(income_list, indent=4)) 
    
#Convert Data to Dataframe
columns = income_data[0] + ["Year"]
rows = income_list[1]

income_df = pd.DataFrame(income_list, columns=columns)
income_df

[
    [
        "Washington County, Oregon",
        "82215",
        "46758",
        "88513",
        "97267",
        "54609",
        "47465",
        "26106",
        "31870",
        "41212",
        "61129",
        "88533",
        "7981",
        "7936",
        "7607",
        "7396",
        "15429",
        "21035",
        "31917",
        "25016",
        "17318",
        "22440",
        "22627",
        "41",
        "067",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "80484",
        "48810",
        "88664",
        "98745",
        "55315",
        "47685",
        "28660",
        "36491",
        "44067",
        "61182",
        "75370",
        "5253",
        "5137",
        "5932",
        "4717",
        "11267",
        "16333",
        "22251",
        "16291",
        "13284",
        "14729",
        "17161",
        "41",
        "005",
        2019
    ],
    [
        "Multnomah County, Oregon",
        "69176",
        "32353",

,NAME,B19013_001E,B19049_002E,B19049_003E,B19049_004E,B19049_005E,B20004_001E,B20004_002E,B20004_003E,B20004_004E,...,B19001_011E,B19001_012E,B19001_013E,B19001_014E,B19001_015E,B19001_016E,B19001_017E,state,county,Year
0,"Washington County, Oregon",82215,46758,88513,97267,54609,47465,26106,31870,41212,...,15429,21035,31917,25016,17318,22440,22627,41,067,2019
1,"Clackamas County, Oregon",80484,48810,88664,98745,55315,47685,28660,36491,44067,...,11267,16333,22251,16291,13284,14729,17161,41,005,2019
2,"Multnomah County, Oregon",69176,32353,77164,76909,49459,42271,24949,30642,35071,...,23949,30866,42859,32603,22187,24555,29154,41,051,2019
3,"Clackamas County, Oregon",82911,49980,90847,102046,59014,48237,30589,37800,44475,...,11225,16165,22667,16928,13259,15748,18524,41,005,2020
4,"Multnomah County, Oregon",71425,34964,79170,80896,52498,43858,25150,31513,36711,...,23902,32123,42399,33680,23477,27166,33788,41,051,2020
5,"Washington County, Oregon",86626,48108,95909,103366,57507,49362,27200,33107,42339,...,15347,21821,30457,27041,19710,24299,24726,41,067,2020
6,"Clackamas County, Oregon",88517,51767,98952,108384,62762,51423,31630,38607,47379,...,9592,14574,22115,17590,13815,17779,21023,41,005,2021
7,"Multnomah County, Oregon",76290,37383,85412,84266,57368,46975,26685,33746,39154,...,22464,32206,43057,34609,24930,31017,38272,41,051,2021
8,"Washington County, Oregon",92025,49559,101803,109991,59965,52966,30095,36271,45054,...,13937,21470,31118,25745,21137,26677,29867,41,067,2021
9,"Clackamas County, Oregon",95740,53544,108275,116104,68201,55711,34581,41203,51076,...,9917,13956,20769,18695,13687,19410,25598,41,005,2022


In [78]:
income_df = income_df.rename(columns={
    "NAME": "County",
    "B19013_001E": "Median Income for all households",
    "B19049_002E": "Median income ages 25 and under",
    "B19049_003E": "Median income ages 25 t0 44",
    "B19049_004E": "Median inome ages 45-64",
    "B19049_005E": "Median income ages 65 +",
    "B20004_001E": "Median earning for all educaiton levels",
    "B20004_002E": "Median earning for less than high school grad",
    "B20004_003E": "Median earning for high school grad",
    "B20004_004E": "Median earning for some college or associates degree",
    "B20004_005E": "Median earning for bachelor's degree",
    "B20004_006E": "Median earning for graduate or professional degree",
    "B19001_007E": "Income $30,000 to $34,999",
    "B19001_008E": "income $35,000 to $39,999",
    "B19001_009E": "Income $40,000 to $44,999",
    "B19001_010E": "Income $45,000 to $49,999",
    "B19001_011E": "Income $50,000 to $59,999",
    "B19001_012E": "Income $60,000 to $74,999",
    "B19001_013E": "Income $75,000 to $99,999",
    "B19001_014E": "Income $100,000 to $124,999",
    "B19001_015E": "Income $125,000 to $149,999",
    "B19001_016E": "Income $150,000 to $199,999",
    "B19001_017E": "Income $200,000 or more"
    })
income_df

,County,Median Income for all households,Median income ages 25 and under,Median income ages 25 t0 44,Median inome ages 45-64,Median income ages 65 +,Median earning for all educaiton levels,Median earning for less than high school grad,Median earning for high school grad,Median earning for some college or associates degree,...,"Income $50,000 to $59,999","Income $60,000 to $74,999","Income $75,000 to $99,999","Income $100,000 to $124,999","Income $125,000 to $149,999","Income $150,000 to $199,999","Income $200,000 or more",state,county,Year
0,"Washington County, Oregon",82215,46758,88513,97267,54609,47465,26106,31870,41212,...,15429,21035,31917,25016,17318,22440,22627,41,067,2019
1,"Clackamas County, Oregon",80484,48810,88664,98745,55315,47685,28660,36491,44067,...,11267,16333,22251,16291,13284,14729,17161,41,005,2019
2,"Multnomah County, Oregon",69176,32353,77164,76909,49459,42271,24949,30642,35071,...,23949,30866,42859,32603,22187,24555,29154,41,051,2019
3,"Clackamas County, Oregon",82911,49980,90847,102046,59014,48237,30589,37800,44475,...,11225,16165,22667,16928,13259,15748,18524,41,005,2020
4,"Multnomah County, Oregon",71425,34964,79170,80896,52498,43858,25150,31513,36711,...,23902,32123,42399,33680,23477,27166,33788,41,051,2020
5,"Washington County, Oregon",86626,48108,95909,103366,57507,49362,27200,33107,42339,...,15347,21821,30457,27041,19710,24299,24726,41,067,2020
6,"Clackamas County, Oregon",88517,51767,98952,108384,62762,51423,31630,38607,47379,...,9592,14574,22115,17590,13815,17779,21023,41,005,2021
7,"Multnomah County, Oregon",76290,37383,85412,84266,57368,46975,26685,33746,39154,...,22464,32206,43057,34609,24930,31017,38272,41,051,2021
8,"Washington County, Oregon",92025,49559,101803,109991,59965,52966,30095,36271,45054,...,13937,21470,31118,25745,21137,26677,29867,41,067,2021
9,"Clackamas County, Oregon",95740,53544,108275,116104,68201,55711,34581,41203,51076,...,9917,13956,20769,18695,13687,19410,25598,41,005,2022


In [79]:
income_df['Income < $50,000'] = income_df[["Income $30,000 to $34,999", "income $35,000 to $39,999", "Income $40,000 to $44,999", "Income $45,000 to $49,999"]].astype(int).sum(axis=1)
income_df['Income $50,000 - $99,999'] = income_df[["Income $50,000 to $59,999", "Income $60,000 to $74,999", "Income $75,000 to $99,999"]].astype(int).sum(axis=1)
income_df['Income $100,000 - $149,999'] = income_df[["Income $100,000 to $124,999", "Income $125,000 to $149,999"]].astype(int).sum(axis=1)
income_df.head()

,County,Median Income for all households,Median income ages 25 and under,Median income ages 25 t0 44,Median inome ages 45-64,Median income ages 65 +,Median earning for all educaiton levels,Median earning for less than high school grad,Median earning for high school grad,Median earning for some college or associates degree,...,"Income $100,000 to $124,999","Income $125,000 to $149,999","Income $150,000 to $199,999","Income $200,000 or more",state,county,Year,"Income < $50,000","Income $50,000 - $99,999","Income $100,000 - $149,999"
0,"Washington County, Oregon",82215,46758,88513,97267,54609,47465,26106,31870,41212,...,25016,17318,22440,22627,41,067,2019,30920,68381,42334
1,"Clackamas County, Oregon",80484,48810,88664,98745,55315,47685,28660,36491,44067,...,16291,13284,14729,17161,41,005,2019,21039,49851,29575
2,"Multnomah County, Oregon",69176,32353,77164,76909,49459,42271,24949,30642,35071,...,32603,22187,24555,29154,41,051,2019,48957,97674,54790
3,"Clackamas County, Oregon",82911,49980,90847,102046,59014,48237,30589,37800,44475,...,16928,13259,15748,18524,41,005,2020,20477,50057,30187
4,"Multnomah County, Oregon",71425,34964,79170,80896,52498,43858,25150,31513,36711,...,33680,23477,27166,33788,41,051,2020,49437,98424,57157


In [80]:
new_income_df = income_df[['County', 'Year', 'Median Income for all households', 'Median income ages 25 and under', 'Median income ages 25 t0 44', 'Median inome ages 45-64', 'Median income ages 65 +', 'Median earning for all educaiton levels', 'Median earning for less than high school grad', 'Median earning for high school grad', 'Median earning for some college or associates degree',"Median earning for bachelor's degree",
    'Median earning for graduate or professional degree', 'Income < $50,000', 'Income $50,000 - $99,999', 'Income $100,000 - $149,999', 'Income $150,000 to $199,999',
    'Income $200,000 or more']]
cleaned_income_df = new_income_df.set_index(['County', 'Year'])
cleaned_income_df.head()


,,Median Income for all households,Median income ages 25 and under,Median income ages 25 t0 44,Median inome ages 45-64,Median income ages 65 +,Median earning for all educaiton levels,Median earning for less than high school grad,Median earning for high school grad,Median earning for some college or associates degree,Median earning for bachelor's degree,Median earning for graduate or professional degree,"Income < $50,000","Income $50,000 - $99,999","Income $100,000 - $149,999","Income $150,000 to $199,999","Income $200,000 or more"
County,Year,,,,,,,,,,,,,,,,
"Washington County, Oregon",2019,82215,46758,88513,97267,54609,47465,26106,31870,41212,61129,88533,30920,68381,42334,22440,22627
"Clackamas County, Oregon",2019,80484,48810,88664,98745,55315,47685,28660,36491,44067,61182,75370,21039,49851,29575,14729,17161
"Multnomah County, Oregon",2019,69176,32353,77164,76909,49459,42271,24949,30642,35071,51406,70511,48957,97674,54790,24555,29154
"Clackamas County, Oregon",2020,82911,49980,90847,102046,59014,48237,30589,37800,44475,62262,78247,20477,50057,30187,15748,18524
"Multnomah County, Oregon",2020,71425,34964,79170,80896,52498,43858,25150,31513,36711,53070,72076,49437,98424,57157,27166,33788


In [110]:
#Export to CSV file
cleaned_income_df.to_csv("output_file/income_data.csv", index=True)


# Housing Costs

In [88]:
#Set Variables
cost_variables = [
    "B25088_001E",  # Total owner-occupied housing units
    "B25088_002E",  # Median monthly owner costs with a mortgage
    # "B25088_003E",  # Median monthly owner costs without a mortgage
    "B25087_005E",  # Owners spending 30%-34.9% of income on housing
    "B25070_007E",  # Renters spending 30%-34.9% of income on rent
    "B25091_017E",  # Owners spending 30%-34.9% of income on costs (with mortgage)
    "B25091_018E",  # Owners spending 35%+ of income on costs (with mortgage)
    "B25001_001E",  # Total Housing Units
    "B25002_002E",  # Occupied Units
    "B25002_003E",  # Vacant Units
    "B25003_002E",  # Owner-occupied Units
    "B25003_003E",  # Renter-occupied Units
    "B25070_010E",  # Renters spending 50%+ of income on rent
    "B25091_020E"  # Owners with a mortgage spending 50%+ of income on housing
    # "B25091_036E"   # Owners without a mortgage spending 50%+ of income on housing
]

years = [2019, 2020, 2021, 2022, 2023]
cost_list = []

#Loop through the years
for year in years:
    #Set the base URL
    url = f"http://api.census.gov/data/{year}/acs/acs5" 
    #Set the Params
    params = {
    "get" : f"NAME,{','.join(cost_variables)}",  #requests a string a variables 
    "for" : "county:005,051,067",   #specific counties Clackamas, Multnomah, Washington
    "in" : "state:41",         #Oregon
    "key" : census_api_key}

# Make the API request
    response = requests.get(url, params=params)
    cost_data = response.json()
    for row in cost_data[1:]:
        row_with_year = row + [year]
        cost_list.append(row_with_year)
    print(json.dumps(cost_list, indent=4)) 

    
    
#Convert Data to Dataframe
columns = cost_data[0] + ["Year"]
rows = cost_list[1]

cost_df = pd.DataFrame(cost_list, columns=columns)
cost_df

[
    [
        "Washington County, Oregon",
        "1693",
        "1972",
        "93",
        "7277",
        "2196",
        "1589",
        "229077",
        "219053",
        "10024",
        "135022",
        "84031",
        "17733",
        "632",
        "41",
        "067",
        2019
    ],
    [
        "Clackamas County, Oregon",
        "1645",
        "2003",
        "14",
        "3604",
        "2313",
        "1569",
        "167104",
        "157408",
        "9696",
        "111885",
        "45523",
        "11498",
        "850",
        "41",
        "005",
        2019
    ],
    [
        "Multnomah County, Oregon",
        "1622",
        "1924",
        "71",
        "12512",
        "3688",
        "2173",
        "348029",
        "326229",
        "21800",
        "177889",
        "148340",
        "37548",
        "1502",
        "41",
        "051",
        2019
    ]
]
[
    [
        "Washington County, Oregon",
        "1693",
        "1972",
  

,NAME,B25088_001E,B25088_002E,B25087_005E,B25070_007E,B25091_017E,B25091_018E,B25001_001E,B25002_002E,B25002_003E,B25003_002E,B25003_003E,B25070_010E,B25091_020E,state,county,Year
0,"Washington County, Oregon",1693,1972,93,7277,2196,1589,229077,219053,10024,135022,84031,17733,632,41,067,2019
1,"Clackamas County, Oregon",1645,2003,14,3604,2313,1569,167104,157408,9696,111885,45523,11498,850,41,005,2019
2,"Multnomah County, Oregon",1622,1924,71,12512,3688,2173,348029,326229,21800,177889,148340,37548,1502,41,051,2019
3,"Clackamas County, Oregon",1683,2067,213,3734,2032,1511,168828,159330,9498,113015,46315,11400,904,41,005,2020
4,"Multnomah County, Oregon",1682,1989,93,13467,4435,2283,353735,334849,18886,182072,152777,38607,1368,41,051,2020
5,"Washington County, Oregon",1725,2024,185,7717,2094,1595,232032,223040,8992,136707,86333,17583,587,41,067,2020
6,"Clackamas County, Oregon",1774,2191,71,3605,2085,1477,169113,159553,9560,113948,45605,12273,924,41,005,2021
7,"Multnomah County, Oregon",1775,2119,109,13363,4520,2350,358675,339228,19447,185493,153735,38253,1340,41,051,2021
8,"Washington County, Oregon",1812,2151,138,8214,2117,1572,234852,226412,8440,138305,88107,17977,674,41,067,2021
9,"Clackamas County, Oregon",1925,2370,65,3485,2330,1543,170723,160984,9739,114452,46532,12534,1070,41,005,2022


In [91]:
cost_df = cost_df.rename(columns={
    "NAME": "County",
    "B25088_001E": "Total owner-occupied housing units",
    "B25088_002E": "Median monthly owner costs with a mortgage",
    "B25087_005E": "Owners spending 30% of income on housing",
    "B25070_007E": "Renters spending 30% of income on rent",
    "B25091_017E": "Owners spending 30%-34.9% of income on costs (with mortgage)",
    "B25091_018E": "Owners spending 35%+ of income on costs (with mortgage)",
    "B25001_001E": "Total Housing Units",
    "B25002_002E": "Occupied Units",
    "B25002_003E": "Vacant Units",
    "B25003_002E": "Owner-occupied Units",
    "B25003_003E": "Renter-occupied Units",
    "B25070_010E": "Renters spending 50%+ of income on rent",
    "B25091_020E": "Owners with a mortgage spending 50%+ of income on housing"
})
cost_df.head()

,County,Total owner-occupied housing units,Median monthly owner costs with a mortgage,Owners spending 30% of income on housing,Renters spending 30% of income on rent,Owners spending 30%-34.9% of income on costs (with mortgage),Owners spending 35%+ of income on costs (with mortgage),Total Housing Units,Occupied Units,Vacant Units,Owner-occupied Units,Renter-occupied Units,Renters spending 50%+ of income on rent,Owners with a mortgage spending 50%+ of income on housing,state,county,Year
0,"Washington County, Oregon",1693,1972,93,7277,2196,1589,229077,219053,10024,135022,84031,17733,632,41,067,2019
1,"Clackamas County, Oregon",1645,2003,14,3604,2313,1569,167104,157408,9696,111885,45523,11498,850,41,005,2019
2,"Multnomah County, Oregon",1622,1924,71,12512,3688,2173,348029,326229,21800,177889,148340,37548,1502,41,051,2019
3,"Clackamas County, Oregon",1683,2067,213,3734,2032,1511,168828,159330,9498,113015,46315,11400,904,41,005,2020
4,"Multnomah County, Oregon",1682,1989,93,13467,4435,2283,353735,334849,18886,182072,152777,38607,1368,41,051,2020


In [95]:
drop_cost_df = cost_df.drop(columns=['state', 'county'])
cleaned_cost_df = drop_cost_df.set_index(["County", "Year"])
cleaned_cost_df.head()

,,Total owner-occupied housing units,Median monthly owner costs with a mortgage,Owners spending 30% of income on housing,Renters spending 30% of income on rent,Owners spending 30%-34.9% of income on costs (with mortgage),Owners spending 35%+ of income on costs (with mortgage),Total Housing Units,Occupied Units,Vacant Units,Owner-occupied Units,Renter-occupied Units,Renters spending 50%+ of income on rent,Owners with a mortgage spending 50%+ of income on housing
County,Year,,,,,,,,,,,,,
"Washington County, Oregon",2019,1693,1972,93,7277,2196,1589,229077,219053,10024,135022,84031,17733,632
"Clackamas County, Oregon",2019,1645,2003,14,3604,2313,1569,167104,157408,9696,111885,45523,11498,850
"Multnomah County, Oregon",2019,1622,1924,71,12512,3688,2173,348029,326229,21800,177889,148340,37548,1502
"Clackamas County, Oregon",2020,1683,2067,213,3734,2032,1511,168828,159330,9498,113015,46315,11400,904
"Multnomah County, Oregon",2020,1682,1989,93,13467,4435,2283,353735,334849,18886,182072,152777,38607,1368


In [134]:
#Export to CSV file
cleaned_cost_df.to_csv("output_file/cost_of_housing.csv", index=True)

# Create one master CSV file

In [97]:
merge1 = pd.merge(cleanded_median_income_df, cleaned_cost_df, on=["County", "Year"])
merge1.head()

,County,Median Household Income,Median Home Value,Median Gross Rent,Year,Total owner-occupied housing units,Median monthly owner costs with a mortgage,Owners spending 30% of income on housing,Renters spending 30% of income on rent,Owners spending 30%-34.9% of income on costs (with mortgage),Owners spending 35%+ of income on costs (with mortgage),Total Housing Units,Occupied Units,Vacant Units,Owner-occupied Units,Renter-occupied Units,Renters spending 50%+ of income on rent,Owners with a mortgage spending 50%+ of income on housing
0,"Washington County, Oregon",82215,386600,1359,2019,1693,1972,93,7277,2196,1589,229077,219053,10024,135022,84031,17733,632
1,"Clackamas County, Oregon",80484,395100,1295,2019,1645,2003,14,3604,2313,1569,167104,157408,9696,111885,45523,11498,850
2,"Multnomah County, Oregon",69176,386200,1237,2019,1622,1924,71,12512,3688,2173,348029,326229,21800,177889,148340,37548,1502
3,"Clackamas County, Oregon",82911,421100,1356,2020,1683,2067,213,3734,2032,1511,168828,159330,9498,113015,46315,11400,904
4,"Multnomah County, Oregon",71425,410800,1309,2020,1682,1989,93,13467,4435,2283,353735,334849,18886,182072,152777,38607,1368


In [98]:
merge2 = pd.merge(merge1, cleaned_income_df, on=["County", "Year"])
merge2.head()

,County,Median Household Income,Median Home Value,Median Gross Rent,Year,Total owner-occupied housing units,Median monthly owner costs with a mortgage,Owners spending 30% of income on housing,Renters spending 30% of income on rent,Owners spending 30%-34.9% of income on costs (with mortgage),...,Median earning for less than high school grad,Median earning for high school grad,Median earning for some college or associates degree,Median earning for bachelor's degree,Median earning for graduate or professional degree,"Income < $50,000","Income $50,000 - $99,999","Income $100,000 - $149,999","Income $150,000 to $199,999","Income $200,000 or more"
0,"Washington County, Oregon",82215,386600,1359,2019,1693,1972,93,7277,2196,...,26106,31870,41212,61129,88533,30920,68381,42334,22440,22627
1,"Clackamas County, Oregon",80484,395100,1295,2019,1645,2003,14,3604,2313,...,28660,36491,44067,61182,75370,21039,49851,29575,14729,17161
2,"Multnomah County, Oregon",69176,386200,1237,2019,1622,1924,71,12512,3688,...,24949,30642,35071,51406,70511,48957,97674,54790,24555,29154
3,"Clackamas County, Oregon",82911,421100,1356,2020,1683,2067,213,3734,2032,...,30589,37800,44475,62262,78247,20477,50057,30187,15748,18524
4,"Multnomah County, Oregon",71425,410800,1309,2020,1682,1989,93,13467,4435,...,25150,31513,36711,53070,72076,49437,98424,57157,27166,33788


In [106]:
final_df = pd.merge(merge2, cleaned_house_age_df, on=["County", "Year"])
final_df = final_df.set_index(['County', 'Year'])
final_df

,,Median Household Income,Median Home Value,Median Gross Rent,Total owner-occupied housing units,Median monthly owner costs with a mortgage,Owners spending 30% of income on housing,Renters spending 30% of income on rent,Owners spending 30%-34.9% of income on costs (with mortgage),Owners spending 35%+ of income on costs (with mortgage),Total Housing Units,...,"Income $200,000 or more",Ages 20-34,Ages 35-44,Ages 45-54,"Owner-occupied, ages 25-34","Owner-occupied, aged 35-44","Owner-occupied, aged 45-54","Renter-occupied, aged 25-34","Renter-occupied, aged 35-44","Renter-occupied, aged 45-54"
County,Year,,,,,,,,,,,,,,,,,,,,,
"Washington County, Oregon",2019,82215,386600,1359,1693,1972,93,7277,2196,1589,229077,...,22627,60380,88671,78489,568,14042,26672,22539,9157,2971
"Clackamas County, Oregon",2019,80484,395100,1295,1645,2003,14,3604,2313,1569,167104,...,17161,34655,53696,55709,392,9226,18341,22288,9267,2728
"Multnomah County, Oregon",2019,69176,386200,1237,1622,1924,71,12512,3688,2173,348029,...,29154,94462,131808,105536,617,17602,38329,29506,12122,4292
"Clackamas County, Oregon",2020,82911,421100,1356,1683,2067,213,3734,2032,1511,168828,...,18524,34934,55216,55252,494,9116,19354,22904,9457,3094
"Multnomah County, Oregon",2020,71425,410800,1309,1682,1989,93,13467,4435,2283,353735,...,33788,95693,133719,106629,597,17728,38400,31387,12380,4804
"Washington County, Oregon",2020,86626,413500,1440,1725,2024,185,7717,2094,1595,232032,...,24726,60784,89993,78772,547,13815,26270,23554,9762,3198
"Clackamas County, Oregon",2021,88517,452200,1447,1774,2191,71,3605,2085,1477,169113,...,21023,34621,57027,55556,475,9077,19987,23679,9698,2990
"Multnomah County, Oregon",2021,76290,437600,1394,1775,2119,109,13363,4520,2350,358675,...,38272,96181,136432,108283,525,18170,39030,32856,12494,4481
"Washington County, Oregon",2021,92025,439300,1541,1812,2151,138,8214,2117,1572,234852,...,29867,60513,91603,79341,595,13980,26739,23762,10232,2953


In [135]:
#Export to CSV file
final_df.to_csv("output_file/Final_DF.csv", index=True)